In [1]:
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
import time
import numpy as np

In [2]:
filepath = "../cleaned_data.csv"

In [3]:
df = pd.read_csv(filepath, nrows=50000)

In [4]:
df

,duration,orig_bytes,resp_bytes,missed_bytes,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,label,proto_icmp,...,conn_state_RSTOS0,conn_state_RSTR,conn_state_RSTRH,conn_state_S0,conn_state_S1,conn_state_S2,conn_state_S3,conn_state_SF,conn_state_SH,conn_state_SHR
0,0.003497,0,0,0.0,5.0,212.0,3.0,144.0,Benign,0,...,0,0,0,0,0,0,0,1,0,0
1,0.036724,34,311,0.0,1.0,62.0,1.0,339.0,Benign,0,...,0,0,0,0,0,0,0,1,0,0
2,384.518261,15072,0,0.0,48.0,16416.0,0.0,0.0,Benign,0,...,0,0,0,1,0,0,0,0,0,0
3,0.270332,48,48,0.0,1.0,76.0,1.0,76.0,Benign,0,...,0,0,0,0,0,0,0,1,0,0
4,0.111429,48,48,0.0,1.0,76.0,1.0,76.0,Benign,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47998,16.002137,212,0,0.0,2.0,268.0,0.0,0.0,Benign,0,...,0,0,0,1,0,0,0,0,0,0
47999,32.008281,318,0,0.0,3.0,402.0,0.0,0.0,Benign,0,...,0,0,0,1,0,0,0,0,0,0
48000,47.719591,385,0,0.0,4.0,497.0,0.0,0.0,Benign,0,...,0,0,0,1,0,0,0,0,0,0
48001,32.008033,318,0,0.0,3.0,402.0,0.0,0.0,Benign,0,...,0,0,0,1,0,0,0,0,0,0


In [5]:
df.shape

(48003, 25)

In [6]:
df['label'].value_counts()

Benign                       26001
PartOfAHorizontalPortScan    12369
C&C                           5618
Attack                        3814
Okiru                          163
DDoS                            36
FileDownload                     2
Name: label, dtype: int64

In [7]:
df.columns = ['duration', 'orig_bytes', 'resp_bytes', 'missed_bytes', 'orig_pkts', 'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes', 'label', 'proto_icmp', 'proto_tcp', 'proto_udp', 'conn_state_OTH', 'conn_state_REJ', 'conn_state_RSTO', 'conn_state_RSTOS0', 'conn_state_RSTR', 'conn_state_RSTRH', 'conn_state_S0', 'conn_state_S1', 'conn_state_S2', 'conn_state_S3', 'conn_state_SF', 'conn_state_SH', 'conn_state_SHR']

In [8]:
for col in df.columns: 
    print(col) 

duration
orig_bytes
resp_bytes
missed_bytes
orig_pkts
orig_ip_bytes
resp_pkts
resp_ip_bytes
label
proto_icmp
proto_tcp
proto_udp
conn_state_OTH
conn_state_REJ
conn_state_RSTO
conn_state_RSTOS0
conn_state_RSTR
conn_state_RSTRH
conn_state_S0
conn_state_S1
conn_state_S2
conn_state_S3
conn_state_SF
conn_state_SH
conn_state_SHR


In [9]:
print(df.columns.tolist())

['duration', 'orig_bytes', 'resp_bytes', 'missed_bytes', 'orig_pkts', 'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes', 'label', 'proto_icmp', 'proto_tcp', 'proto_udp', 'conn_state_OTH', 'conn_state_REJ', 'conn_state_RSTO', 'conn_state_RSTOS0', 'conn_state_RSTR', 'conn_state_RSTRH', 'conn_state_S0', 'conn_state_S1', 'conn_state_S2', 'conn_state_S3', 'conn_state_SF', 'conn_state_SH', 'conn_state_SHR']


In [10]:
X = df[['duration', 'orig_bytes', 'resp_bytes', 'missed_bytes', 'orig_pkts', 'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes', 'proto_icmp', 'proto_tcp', 'proto_udp', 'conn_state_OTH', 'conn_state_REJ', 'conn_state_RSTO', 'conn_state_RSTOS0', 'conn_state_RSTR', 'conn_state_RSTRH', 'conn_state_S0', 'conn_state_S1', 'conn_state_S2', 'conn_state_S3', 'conn_state_SF', 'conn_state_SH', 'conn_state_SHR']]
Y = df['label']

In [11]:
SVM_classifier = SVC()
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=10, test_size=0.2)

In [12]:
start = time.time()

SVM_classifier = SVC(C=1.0, cache_size=1500, verbose=True)
SVM_classifier.fit(X_train, Y_train)
print()
print(SVM_classifier.score(X_test, Y_test))
print()

y_pred = SVM_classifier.predict(X_test)
print(y_pred)
print()

end = time.time()
print('time cost: ',end - start, 'seconds')

[LibSVM]
0.5387980418706385

['Benign' 'Benign' 'Benign' ... 'Benign' 'Benign' 'Benign']

time cost:  101.04595017433167 seconds


In [13]:
print("Classifiction Report :")
print(classification_report(Y_test, y_pred))

Classifiction Report :


c:\Users\JAYDIP\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\JAYDIP\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\JAYDIP\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(avera

                           precision    recall  f1-score   support

                   Attack       0.00      0.00      0.00       754
                   Benign       0.54      1.00      0.70      5169
                      C&C       0.00      0.00      0.00      1146
                     DDoS       1.00      0.50      0.67         8
                    Okiru       0.00      0.00      0.00        32
PartOfAHorizontalPortScan       0.00      0.00      0.00      2492

                 accuracy                           0.54      9601
                macro avg       0.26      0.25      0.23      9601
             weighted avg       0.29      0.54      0.38      9601

